# Thermal Expansion Lab

Usual error method library.

In [1]:
import numpy as np
from typing import Iterable, Tuple, Union

# Define our rules...

def rule_1(c, error_val):
    return abs(c) * error_val

def rule_2(c, val, error_val, power):
    return abs(c * power * val ** (power - 1)) * error_val

def rule_3(*err_vals: Iterable[float]) -> float:
    """
    Calculate rule 3 from the paper 'Treatment of Data'.
    
    @param err_vals: A list of parameters, being the errors in each value. It is assumed they were summed 
                     together to get the final value.
    
    @returns: The error of all of the sum of the values...
    """
    total = 0
    
    for err in err_vals:
        total += err ** 2
    
    return np.sqrt(total)


def rule_4(value: float, *error_list: Iterable[Tuple[float, float, float]]) -> float:
    """
    Calculate rule 4 from the paper 'Treatment of Data'.
    
    @param value: The value of the thing we are trying to calculate the error of.
    @param error_list: A list of length 3 tuples. 
                       Each tuple should contain:
                       - A float: A value in the error formula.
                       - A float: The measured error in the above value.
                       - A float: The power of the above value in the multiplicative formula.

    @returns: A float, being the error in 'value'.
    """
    total = 0
    
    for x, x_err, power in error_list:
        total += (power * (x_err / x)) ** 2
        
    return abs(value) * np.sqrt(total)

# Fixed bug in checking code below...
FloatVec = Union[float, np.ndarray] 
BoolVec = Union[bool, np.ndarray]

def values_agree(val_1: FloatVec, err_1: FloatVec, val_2: FloatVec, err_2: FloatVec) -> BoolVec:
    """
    Determines if the values in 2 vectors(arrays) agree with each other, given there uncertainty values. 
    
    @param val_1: The 1st array of values.
    @param err_1: The uncertainty values for the 1st vector.
    @param val_2: The 2nd array of values.
    @param err_2: The uncertainty values for the 2st vector. 
    
    @returns: A vector of booleans, being whether each value agrees with the other.
    """
    # Grab the ranges for each value...
    r11, r12 = val_1 - err_1, val_1 + err_1
    r21, r22 = val_2 - err_2, val_2 + err_2
    
    # The ranges are sorted (r21 <= r22 and r11 <= r12), so the simple 2 checks below are enough. 
    # Check 1: Does r21(lowest value of 2nd range) land above the 1st range? If so fail...
    # Check 2: Does r22(highest value of 2nd range) fall below the 1st range? If so fail...
    return ((r21 <= r12) & (r11 <= r22))

## Pull in Data and Compute Coefficients of Linear Expansion

In [36]:
import pandas as pd

data = pd.read_csv("pipedata.csv", index_col=0)

dL = (data["Dial (final) (mm)"] / 1000) - (data["Dial (init) (mm)"] / 1000)
err_dL = rule_3(data["Err Dial (mm)"] / 1000, data["Err Dial (mm)"] / 1000)

dT = data["T (final) (C )"] - data["T (init) (C )"]
err_dT = rule_3(data["Err T (C )"], data["Err T (C )"])

L = data["L (cm)"] / 100
err_L = data["Err L (mm)"] / 1000

alpha = dL / (L * dT)
err_alpha = rule_4(alpha, (dL, err_dL, 1), (L, err_L, -1), (dT, err_dT, -1))

print("Alpha:")
display(alpha.to_frame().T)
print("Error in Alpha:")
display(err_alpha.to_frame().T)

Alpha:


,Aluminium Tube,Copper Tube
0,0.000028,0.000019


Error in Alpha:


,Aluminium Tube,Copper Tube
0,3.887604e-07,3.450446e-07
